# Tarea 6: La falla del nodo 214

In [16]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import networkx as nx

from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.statespace.sarimax import SARIMAX


PATHS = {
    "energy_csv": "data/ener_clean.csv",   
    "agro_csv": "data/agro_clean.csv",     
}

dfE = pd.read_csv(PATHS["energy_csv"])
dfA = pd.read_csv(PATHS["agro_csv"])

print("Energy shape:", dfE.shape)
print("Agro shape:", dfA.shape)
print(dfE.columns)
print(dfA.columns)

def ensure_sorted_by_time(df: pd.DataFrame, time_col_candidates=None):
    """
    Intenta ordenar por una columna temporal si existe.
    Si no existe, deja el orden como está (asume que ya está ordenado).
    """
    if time_col_candidates is None:
        time_col_candidates = ["timestamp", "date", "datetime", "time", "Fecha", "fecha"]

    for c in time_col_candidates:
        if c in df.columns:
            df = df.copy()
            df[c] = pd.to_datetime(df[c], errors="coerce")
            df = df.sort_values(c)
            return df
    return df


def adf_test(series: pd.Series, name="series"):
    """
    ADF test con impresión clara.
    """
    s = series.dropna()
    if len(s) < 20:
        return {"name": name, "ok": False, "msg": "Muy pocos datos para ADF."}

    stat, pval, usedlag, nobs, crit, icbest = adfuller(s, autolag="AIC")
    return {
        "name": name,
        "ok": True,
        "adf_stat": stat,
        "pvalue": pval,
        "usedlag": usedlag,
        "nobs": nobs,
        "crit": crit,
        "stationary_5pct": (pval < 0.05),
    }


def build_directed_graph_from_edges(df: pd.DataFrame, source_col="Source_Node", target_col="Target_Node"):
    """
    Construye grafo dirigido desde columnas Source_Node y Target_Node.
    Si hay repetidos, lo maneja agregando peso por conteo.
    """
    edges = df[[source_col, target_col]].dropna().copy()
    # Conteo de aristas repetidas como weight
    edge_counts = edges.value_counts().reset_index(name="weight")

    G = nx.DiGraph()
    for _, row in edge_counts.iterrows():
        u = row[source_col]
        v = row[target_col]
        w = row["weight"]
        G.add_edge(u, v, weight=float(w))
    return G


def safe_granger(df2: pd.DataFrame, maxlag=4, verbose=True):
    """
    Ejecuta Granger con manejo de errores.
    df2 debe tener dos columnas en orden [y, x] donde:
      - pregunta: "x causa y?"
      - grangercausalitytests evalúa si x ayuda a predecir y (primera col).
    """
    df2 = df2.dropna()
    if len(df2) < (maxlag + 5):
        raise ValueError("Muy pocos datos para Granger con ese maxlag.")

    results = grangercausalitytests(df2, maxlag=maxlag, verbose=verbose)

    # Resumen: tomar p-values del test ssr_ftest por lag
    summary = []
    for lag in range(1, maxlag + 1):
        pval = results[lag][0]["ssr_ftest"][1]
        summary.append({"lag": lag, "pvalue_ssr_ftest": pval})
    return pd.DataFrame(summary)


def fit_arimax(endog: pd.Series, exog: pd.DataFrame, order=(1,0,1), enforce_stationarity=False, enforce_invertibility=False):
    """
    Ajusta SARIMAX como ARIMAX (sin estacionalidad).
    """
    endog = endog.astype(float)
    exog = exog.astype(float)

    model = SARIMAX(
        endog=endog,
        exog=exog,
        order=order,
        enforce_stationarity=enforce_stationarity,
        enforce_invertibility=enforce_invertibility
    )
    res = model.fit(disp=False)
    return res


Energy shape: (2000, 14)
Agro shape: (2000, 14)
Index(['Ener_1', 'Ener_2', 'Ener_3', 'Ener_4', 'Ener_5', 'Ener_6', 'Ener_7',
       'Ener_8', 'Ener_9', 'Ener_10', 'Latitude', 'Longitude', 'Source_Node',
       'Target_Node'],
      dtype='object')
Index(['Agro_1', 'Agro_2', 'Agro_3', 'Agro_4', 'Agro_5', 'Agro_6', 'Agro_7',
       'Agro_8', 'Agro_9', 'Agro_10', 'Latitude', 'Longitude', 'Source_Node',
       'Target_Node'],
      dtype='object')


### P1: Causalidad y Redes

In [21]:
# Ordenar por tiempo si aplica
dfE = ensure_sorted_by_time(dfE)

# Grafo dirigido y centralidades
G = build_directed_graph_from_edges(dfE, "Source_Node", "Target_Node")

deg_centrality = nx.degree_centrality(G)  
bet_centrality = nx.betweenness_centrality(G, normalized=True, weight=None)  

# Nodo cuello de botella:
bottleneck_node = max(bet_centrality, key=bet_centrality.get)
print("\n--- GRAFO ---")
print("Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())
print("Bottleneck node (max betweenness):", bottleneck_node, "| betweenness:", bet_centrality[bottleneck_node])

# Granger: Factor de Potencia (Ener_10) -> Voltaje (Ener_9)
print("\n--- P1: GRANGER (Ener_10 -> Ener_9) ---")
granger_df = dfE[["Ener_9", "Ener_10"]].copy()
granger_df = granger_df.replace([np.inf, -np.inf], np.nan).dropna()

# Ejecuta test
granger_summary = safe_granger(granger_df[["Ener_9", "Ener_10"]], maxlag=4, verbose=False)
print(granger_summary)

# Interpretación simple:
if (granger_summary["pvalue_ssr_ftest"] < 0.05).any():
    print("✅ Hay evidencia de causalidad de Granger: Ener_10 ayuda a predecir Ener_9 (en al menos un lag).")
else:
    print("❌ No hay evidencia suficiente de Granger (p>=0.05 en todos los lags).")

# Cómo afectaría un fallo del nodo con mayor betweenness:
print("\n--- IMPACTO SIMULADO: remover bottleneck ---")
G_removed = G.copy()
G_removed.remove_node(bottleneck_node)

# Componentes fuertemente conexas (en digrafo)
scc_before = nx.number_strongly_connected_components(G)
scc_after = nx.number_strongly_connected_components(G_removed)

print("Strongly connected components BEFORE:", scc_before)
print("Strongly connected components AFTER :", scc_after)
print("➡️ Si sube mucho después de remover, la red queda más fragmentada (más riesgo de interrupciones).")



--- GRAFO ---
Nodes: 70 Edges: 865
Bottleneck node (max betweenness): 119 | betweenness: 0.0

--- P1: GRANGER (Ener_10 -> Ener_9) ---
   lag  pvalue_ssr_ftest
0    1          0.302454
1    2          0.342283
2    3          0.430454
3    4          0.019318
✅ Hay evidencia de causalidad de Granger: Ener_10 ayuda a predecir Ener_9 (en al menos un lag).

--- IMPACTO SIMULADO: remover bottleneck ---
Strongly connected components BEFORE: 70
Strongly connected components AFTER : 69
➡️ Si sube mucho después de remover, la red queda más fragmentada (más riesgo de interrupciones).


### P2: Optimización Geo-Agrónoma

In [ ]:
print("\n--- P2: GEO-AGRO ---")
dfA = ensure_sorted_by_time(dfA)

# Suavizado simple para "filtrar ruido GPS" (rolling mean)
# Ajusta ventana según densidad: 10 es un buen inicio.
gps_window = 10
dfA["Latitude_f"]  = dfA["Latitude"].rolling(gps_window, min_periods=1).mean()
dfA["Longitude_f"] = dfA["Longitude"].rolling(gps_window, min_periods=1).mean()

# (B) "Pendiente" proxy sin DEM:
# Usamos magnitud del gradiente espacial (aprox) como indicador de cambio brusco del terreno/relieve
# Nota: en un trabajo real usarías DEM (SRTM, etc.), pero aquí el reto dice "asuma relación..."
dlat = dfA["Latitude_f"].diff()
dlon = dfA["Longitude_f"].diff()
dfA["slope_proxy"] = np.sqrt(dlat**2 + dlon**2).fillna(0)

# (C) Varianza del viento (Agro_10) en ventana móvil
wind_window = 50
dfA["wind_var"] = dfA["Agro_10"].rolling(wind_window, min_periods=10).var()

# (D) Identificar sensores de menor NDVI (Agro_5)
# Regla práctica: bottom 10% como "bajo NDVI"
ndvi = dfA["Agro_5"].replace([np.inf, -np.inf], np.nan)
threshold_ndvi = ndvi.quantile(0.10)

low_ndvi = dfA[dfA["Agro_5"] <= threshold_ndvi].copy()

# (E) Validar si caen en zona de alta pendiente y alta varianza de viento
slope_thr = dfA["slope_proxy"].quantile(0.75)
wind_thr  = dfA["wind_var"].quantile(0.75)

low_ndvi["high_slope"] = low_ndvi["slope_proxy"] >= slope_thr
low_ndvi["high_windvar"] = low_ndvi["wind_var"] >= wind_thr

print("NDVI threshold (10%):", threshold_ndvi)
print("Low NDVI count:", len(low_ndvi))
print("Proporción en alta pendiente:", low_ndvi["high_slope"].mean())
print("Proporción en alta varianza de viento:", low_ndvi["high_windvar"].mean())

# (F) Recomendación automática (texto base)
recommendation = []
if low_ndvi["high_slope"].mean() > 0.5 and low_ndvi["high_windvar"].mean() > 0.5:
    recommendation.append(
        "Los sensores de bajo NDVI se concentran en zonas con alta pendiente y alta varianza de viento: "
        "recomienda inversión en infraestructura hídrica (riego por goteo, micro-reservorios) + rompevientos."
    )
else:
    recommendation.append(
        "No hay evidencia fuerte de que bajo NDVI coincida masivamente con alta pendiente/varianza de viento: "
        "recomienda validar microzonas y priorizar inversión donde sí coincidan condiciones de estrés."
    )
print("\nRECOMENDACIÓN P2:", recommendation[0])




--- P2: GEO-AGRO ---
NDVI threshold (10%): 0.7811757380105776
Low NDVI count: 200
Proporción en alta pendiente: 0.315
Proporción en alta varianza de viento: 0.27

RECOMENDACIÓN P2: No hay evidencia fuerte de que bajo NDVI coincida masivamente con alta pendiente/varianza de viento: recomienda validar microzonas y priorizar inversión donde sí coincidan condiciones de estrés.


### P3: Analítica Predictiva

In [11]:
# (A) Construir exógenas: Temperatura (Ener_3) y centralidad del Source_Node
# Centralidad: aquí usamos betweenness (puedes cambiar a degree si quieres)
dfE["centrality"] = dfE["Source_Node"].map(bet_centrality)

# (B) Preparar series
endog_raw = dfE["Ener_1"].replace([np.inf, -np.inf], np.nan)
exog_base = dfE[["Ener_3"]].replace([np.inf, -np.inf], np.nan)
exog_plus = dfE[["Ener_3", "centrality"]].replace([np.inf, -np.inf], np.nan)

# (C) Estacionariedad: si no es estacionaria, diferenciar
adf_endog = adf_test(endog_raw, "Ener_1")
print("ADF Ener_1:", adf_endog)

if adf_endog.get("stationary_5pct", False):
    endog = endog_raw
    exog1 = exog_base
    exog2 = exog_plus
else:
    endog = endog_raw.diff()
    exog1 = exog_base.iloc[1:].copy()
    exog2 = exog_plus.iloc[1:].copy()

# Alinear NaNs
df_model1 = pd.concat([endog, exog1], axis=1).dropna()
df_model2 = pd.concat([endog, exog2], axis=1).dropna()

endog1 = df_model1.iloc[:, 0]
X1 = df_model1.iloc[:, 1:]

endog2 = df_model2.iloc[:, 0]
X2 = df_model2.iloc[:, 1:]

# (D) Ajuste ARIMAX (elige un order base, puedes tunear con grid si quieres)
order = (1, 0, 1)

res1 = fit_arimax(endog1, X1, order=order)
res2 = fit_arimax(endog2, X2, order=order)

print("AIC sin centralidad:", res1.aic)
print("AIC con centralidad:", res2.aic)

if res2.aic < res1.aic:
    print("✅ Incluir centralidad mejora el AIC (mejor ajuste).")
else:
    print("❌ No mejora el AIC al incluir centralidad (con este order).")

# (E) Mostrar coeficientes
print("\n--- COEFICIENTES MODELO SIN CENTRALIDAD ---")
print(res1.params)

print("\n--- COEFICIENTES MODELO CON CENTRALIDAD ---")
print(res2.params)


ADF Ener_1: {'name': 'Ener_1', 'ok': True, 'adf_stat': -1.866891087440951, 'pvalue': 0.3477932313370453, 'usedlag': 26, 'nobs': 1973, 'crit': {'1%': -3.4336687168076714, '5%': -2.863006019389988, '10%': -2.567550447906854}, 'stationary_5pct': False}
AIC sin centralidad: 8749.99211736723
AIC con centralidad: 8751.992117367226
❌ No mejora el AIC al incluir centralidad (con este order).

--- COEFICIENTES MODELO SIN CENTRALIDAD ---
Ener_3   -0.000680
ar.L1    -0.054475
ma.L1    -0.762763
sigma2    4.661078
dtype: float64

--- COEFICIENTES MODELO CON CENTRALIDAD ---
Ener_3       -0.000680
centrality    0.000000
ar.L1        -0.054475
ma.L1        -0.762763
sigma2        4.661078
dtype: float64


### Informe

In [12]:
outputs = {
    "bottleneck_node": bottleneck_node,
    "bottleneck_betweenness": float(bet_centrality[bottleneck_node]),
    "granger_summary": granger_summary,
    "scc_before": int(scc_before),
    "scc_after": int(scc_after),
    "ndvi_threshold_10pct": float(threshold_ndvi) if pd.notna(threshold_ndvi) else None,
    "low_ndvi_count": int(len(low_ndvi)),
    "low_ndvi_high_slope_ratio": float(low_ndvi["high_slope"].mean()) if len(low_ndvi) else None,
    "low_ndvi_high_windvar_ratio": float(low_ndvi["high_windvar"].mean()) if len(low_ndvi) else None,
    "arimax_aic_no_centrality": float(res1.aic),
    "arimax_aic_with_centrality": float(res2.aic),
    "p2_recommendation": recommendation[0],
}

print("\n==================== RESUMEN PARA INFORME ====================")
for k, v in outputs.items():
    print(k, ":", v)


==================== RESUMEN PARA INFORME ====================
bottleneck_node : 119
bottleneck_betweenness : 0.0
granger_summary :    lag  pvalue_ssr_ftest
0    1          0.302454
1    2          0.342283
2    3          0.430454
3    4          0.019318
scc_before : 70
scc_after : 69
ndvi_threshold_10pct : 0.7811757380105776
low_ndvi_count : 200
low_ndvi_high_slope_ratio : 0.315
low_ndvi_high_windvar_ratio : 0.27
arimax_aic_no_centrality : 8749.99211736723
arimax_aic_with_centrality : 8751.992117367226
p2_recommendation : No hay evidencia fuerte de que bajo NDVI coincida masivamente con alta pendiente/varianza de viento: recomienda validar microzonas y priorizar inversión donde sí coincidan condiciones de estrés.
